In [ ]:
import meshio as mio
import numpy as np
import os
import sys
import json
import subprocess
import glob

input_file_path = "orig"
executables_json = "executables.json"
normalized_path = "normalized"
embedded_path = "embedded"

In [ ]:
def run_command(command):
    print("===== Execute:", command)
    try:
        subprocess.run(command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"!!!!! Command '{command}' failed with error: {e} !!!!!")

In [ ]:
executables_json = "executables.json"

# read executables json
with open(executables_json, "r") as f:
    executables = json.load(f)

wmtk_tetwild_binary = executables["wmtk_tetwild_binary"]
wmtk_msh_converter_binary = executables["wmtk_msh_converter_binary"]

In [ ]:
# mesh file names
input_files = [
    os.path.join(input_file_path, file)
    for file in os.listdir(input_file_path)
    if file.endswith(".off")
]
input_files

### Normalize Models

In [ ]:
os.makedirs(normalized_path, exist_ok=True)

for f in input_files:
    f_no_ext = os.path.basename(f)
    name = os.path.splitext(f_no_ext)[0]
    
    print(name)
    m = mio.read(f)
    pts = m.points
    p0 = pts.min(axis=0)
    p1 = pts.max(axis=0)
    sf = (p1 - p0).max()
    # print("Pts orig:\n", pts)
    print("\tMin:", p0)
    print("\tMax:", p1)
    print("\tScaling factor:", sf)
    pts -= p0
    pts /= sf
    # print("Pts:\n", pts)


    m.write(
        os.path.join(normalized_path, name + ".msh"),
        file_format="gmsh",
        binary=True,
    )
    m.write(os.path.join(normalized_path, name + ".obj"))

In [ ]:
# normalized mesh file names
normalized_files = [
    os.path.abspath(os.path.join(normalized_path, file))
    for file in os.listdir(normalized_path)
    if file.endswith(".msh")
]
normalized_files

### TetWild

In [ ]:
tetwild_base_json = {
    "input": "file",
    "output": "out",
    "envelope_size": 1e-2,
    "target_edge_length": 0.2,
}

In [ ]:
# Generate folders and JSON files
os.makedirs(embedded_path, exist_ok=True)

tetwild_jsons = []

for f in normalized_files:
    f_no_ext = os.path.basename(f)
    name = os.path.splitext(f_no_ext)[0]
    
    os.makedirs(os.path.join(embedded_path, name), exist_ok=True)
    
    tw = tetwild_base_json.copy()
    tw['input'] = f
    tw['output'] = name
    
    tw_json = json.dumps(tw, indent=4)
    
    tw_json_file = os.path.join(embedded_path, name, name + ".json")

    with open(tw_json_file, "w") as j:
        j.write(tw_json)
    
    tetwild_jsons.append(os.path.abspath(tw_json_file))

tetwild_jsons

In [ ]:
# Run tetwild
commands = []

for tw_json in tetwild_jsons:
    exec_path = os.path.dirname(tw_json)
    # print(exec_path)

    command = " ".join(
        [
            "cd",
            exec_path,
            ";",
            wmtk_tetwild_binary,
            "-j",
            tw_json,
        ]
    )
    commands.append(command)

commands

In [ ]:
for command in commands:
    run_command(command)

In [60]:
# clean up
for tw_json in tetwild_jsons:
    exec_path = os.path.dirname(tw_json)

    f_no_ext = os.path.basename(tw_json)
    name = os.path.splitext(f_no_ext)[0]

    for f in glob.glob(os.path.join(exec_path, "*.vtu")):
        if os.path.basename(f) != name + "_tets.vtu":
            os.remove(f)

    for f in glob.glob(os.path.join(exec_path, "*.vtm")):
        os.remove(f)

    m = mio.read(os.path.join(exec_path, name + "_tets.vtu"))

    # make winding number positive
    a = m.cell_data["winding_number"][0].copy()
    if np.max(a) > 0.5:
        a = np.max(a) - a
    else:
        a = a - np.min(a)
    m.cell_data["winding_number"] = a[:, None]

    msh_file_name = os.path.join(exec_path, name + "_tets.msh")
    print("Write", msh_file_name)
    m.write(msh_file_name, file_format="gmsh", binary=True)

Write /Users/dz2425/projects/algebraic-contours/examples/input/input_stls/embedded/cube/cube_tets.msh


### Make Winding Number Positive